# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#import turicreate as tc
from sklearn.preprocessing import StandardScaler

#### Data Import

In [2]:
%%time

#Loading all data file separately 
vle = pd.read_csv('./anonymiseddata/vle.csv')
assessments = pd.read_csv('./anonymiseddata/assessments.csv')
courses = pd.read_csv('./anonymiseddata/courses.csv')
studentAssessments = pd.read_csv('./anonymiseddata/studentAssessment.csv')
studentInfo = pd.read_csv('./anonymiseddata/studentInfo.csv')
studentRegistration = pd.read_csv('./anonymiseddata/studentRegistration.csv')
studentVle = pd.read_csv('./anonymiseddata/studentVle.csv')

CPU times: user 3.7 s, sys: 672 ms, total: 4.37 s
Wall time: 4.47 s


# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

For Lab 3 we chose to use the "Open University Learning Analytics Dataset" (OULAD) which is an anonymized collection of information acquired from online students who were enrolled in the Open University (Milton Keynes, England) in 2013-2014. The data includes student demographic and registration information, as well as assessment results and interactions with the Virtual Learning Environment (VLE). There are multiple ways that the OULAD can be utilized, including prediction of grade results, better understanding of the factors that influence online student outcomes, or to compare online student results to in-person student results. The reason why this dataset is important is because it can be used by educators and universities to increase student success within an online learning environment. 

Our goal for Lab 3 was to explore the OULAD using K-Means clustering in order to find trends and insight into student learning styles. We chose K-Means clustering because it is an unsupervised algorithm that is straightforward to carry out, scalable, and can be used for large datasets. We chose to evaluate the clustering model using the **Dunn Index..?**, which calculates the closest distance between two clusters divided by the largest distance between two clusers. The Dunn Index has a range of 0 to 1, wherein the closer the value is to 1 the better the clustering. We felt that the Dunn Index was ideal because it provides a numeric value to the fitness of a cluster rather than through visual analysis.


##### References
[Kaggle: OULAD](https://www.kaggle.com/datasets/rocki37/open-university-learning-analytics-dataset)
###### https://mayankdw.medium.com/k-means-clustering-and-dunn-index-implementaion-from-scratch-9c66573bfe90
###### https://python-bloggers.com/2022/03/dunn-index-for-k-means-clustering-evaluation/
###### https://www.analyticsvidhya.com/blog/2020/10/a-simple-explanation-of-k-means-clustering/

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

The OULAD contains seven CSV files that encompasses data collected from 32,593 students that were enrolled in 22 courses. There are a total of 28 variables and over 1 million instances. The CSV files and their contents are described below, information regarding each variable was taken from [Here](https://analyse.kmi.open.ac.uk/open_dataset). It should be noted that "modules" refer to university courses.

#### Courses:
* code_module: code for the module, which serves as the identifier
* code_presentation – code name of the presentation. It consists of the year and “B” for the presentation starting in February and “J” for the presentation starting in October.
* length - length of the module-presentation in days.

#### Assessments
* code_module – identification code of the module, to which the assessment belongs.
* code_presentation - identification code of the presentation, to which the assessment belongs.
* id_assessment – identification number of the assessment.
* assessment_type – type of assessment. Three types of assessments exist: Tutor Marked Assessment (TMA), Computer Marked Assessment (CMA) and Final Exam (Exam).
* date – information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero).
* weight - weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%.

#### VLE
* id_site – an identification number of the material.
* code_module – an identification code for module.
* code_presentation - the identification code of presentation.
* activity_type – the role associated with the module material.
* week_from – the week from which the material is planned to be used.
* week_to – week until which the material is planned to be used.;

In [18]:
vle.activity_type.value_counts()

resource          2660
subpage           1055
oucontent          996
url                886
forumng            194
quiz               127
page               102
oucollaborate       82
questionnaire       61
ouwiki              49
dataplus            28
externalquiz        26
homepage            22
ouelluminate        21
glossary            21
dualpane            20
repeatactivity       5
htmlactivity         4
sharedsubpage        3
folder               2
Name: activity_type, dtype: int64

#### StudentInfo
* code_module – an identification code for a module on which the student is registered.
* code_presentation - the identification code of the presentation during which the student is registered on the module.
* id_student – a unique identification number for the student.
* gender – the student’s gender.
* region – identifies the geographic region, where the student lived while taking the module-presentation.
* highest_education – highest student education level on entry to the module presentation.
* imd_band – specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.
* age_band – band of the student’s age.
* num_of_prev_attempts – the number times the student has attempted this module.
* studied_credits – the total number of credits for the modules the student is currently studying.
* disability – indicates whether the student has declared a disability.
* final_result – student’s final result in the module-presentation.

#### StudentRegistragion
* code_module – an identification code for a module.
* code_presentation - the identification code of the presentation.
* id_student – a unique identification number for the student.
* date_registration – the date of student’s registration on the module presentation, this is the number of days measured relative to the start of the module-presentation (e.g. the negative value -30 means that the student registered to module presentation 30 days before it started).
* date_unregistration – date of student unregistration from the module presentation, this is the number of days measured relative to the start of the module-presentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file.

#### StudentAssessment
* id_assessment – the identification number of the assessment.
* id_student – a unique identification number for the student.
* date_submitted – the date of student submission, measured as the number of days since the start of the module presentation.
* is_banked – a status flag indicating that the assessment result has been transferred from a previous presentation.
* score – the student’s score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.

#### StudentVLE
* code_module – an identification code for a module.
* code_presentation - the identification code of the module presentation.
* id_student – a unique identification number for the student.
* id_site - an identification number for the VLE material.
* date – the date of student’s interaction with the material measured as the number of days since the start of the module-presentation.
* sum_click – the number of times a student interacts with the material in that day.



### Data Description

 The data comes in through a relational database schema with separated tables. We needed to merge these tables together into a singular dataframe
 
 - The data set appears to be for the UK
 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
  
  
  we found that using all of the 32000 students was too intensive, so we sampled 4000 random students to create the dataframe from

In [3]:
%%time
# Randomly sampling 4,000 records StudentVle Table with Million Rows.
studentSample = studentInfo.sample(n=1000, random_state = 777)
print(studentSample.shape)

# vle
# assessments X
# courses X
# studentAssessments X
# studentInfo X
# studentRegistration X
# studentVleSample X

df = pd.merge(studentSample, studentVle, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment', indicator=True)
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, courses, on= ['code_module', 'code_presentation'], how='left')
df = pd.merge(df, studentRegistration, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, vle, on=['id_site', 'code_module', 'code_presentation'], how='left')
print(df.shape)
df.head()

(1000, 12)
(2705197, 29)
CPU times: user 4.43 s, sys: 1.35 s, total: 5.79 s
Wall time: 5.92 s


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,assessment_type,date_y,weight,_merge,module_presentation_length,date_registration,date_unregistration,activity_type,week_from,week_to
0,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,CMA,18.0,2.0,both,269,-70.0,32.0,homepage,NaN,NaN
1,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,CMA,18.0,2.0,both,269,-70.0,32.0,resource,NaN,NaN
2,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,CMA,18.0,2.0,both,269,-70.0,32.0,forumng,NaN,NaN
3,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,CMA,18.0,2.0,both,269,-70.0,32.0,forumng,NaN,NaN
4,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,CMA,18.0,2.0,both,269,-70.0,32.0,homepage,NaN,NaN


making sure we are not creating rows with missing values

In [4]:
df._merge.value_counts()
df = df.drop(columns='_merge')

### Missing Values

In [5]:
print('Missing values: \n', df.isnull().sum() * 100 / len(df),'\n\n')
df = df.drop(columns = ['week_from', 'week_to', 'date_unregistration'])
print('Missing values after dopping columns: \n', df.isnull().sum() * 100 / len(df))
print('\n\nTotal percentage of missing values: ', df.isnull().sum().sum() * 100 / len(df))
print('\n\nTotal missing values: ', df.isnull().sum().sum())

Missing values: 
 code_module                    0.000000
code_presentation              0.000000
id_student                     0.000000
gender                         0.000000
region                         0.000000
highest_education              0.000000
imd_band                       4.815878
age_band                       0.000000
num_of_prev_attempts           0.000000
studied_credits                0.000000
disability                     0.000000
final_result                   0.000000
id_site                        0.000000
date_x                         0.000000
sum_click                      0.000000
id_assessment                  0.138437
date_submitted                 0.138437
is_banked                      0.138437
score                          0.191816
assessment_type                0.138437
date_y                         1.902227
weight                         0.138437
module_presentation_length     0.000000
date_registration              0.000000
date_unregistration   

From the cell above, we went ahead and dropped the week_from, week_to, and date_unregistration columns as these all were mostly missing values. These columns that were dropped could be used to analyze the population of the data that are not registered.

Below we can analyze the missing values for IMD Band and potentially fill those with values. Otherwise we need to justify dropping these rows.

In [6]:
missingIMDBand = df[df['imd_band'].isnull()]

print('counts per region: \n',missingIMDBand.region.value_counts(),'\n\n')
print('percentage breakdown per region: \n',missingIMDBand.region.value_counts() * 100 / len(missingIMDBand), '\n\n')

northRegionMV = missingIMDBand.region.value_counts()[0]
irelandMV = missingIMDBand.region.value_counts()[1]
westMidRegionMV = missingIMDBand.region.value_counts()[2]
southRegionMV = missingIMDBand.region.value_counts()[3]

print('North Region percent of total: ',northRegionMV/len(df.region[df.region=='North Region']))
print('Ireland percent of total: ',irelandMV/len(df.region[df.region=='Ireland']))
print('West Midlands Region percent of total: ',westMidRegionMV/len(df.region[df.region=='West Midlands Region']))
print('South Region percent of total: ',southRegionMV/len(df.region[df.region=='South Region']))

counts per region: 
 North Region            84879
Ireland                 32446
South Region            11856
West Midlands Region     1098
Name: region, dtype: int64 


percentage breakdown per region: 
 North Region            65.151713
Ireland                 24.905012
South Region             9.100469
West Midlands Region     0.842807
Name: region, dtype: float64 


North Region percent of total:  0.5453932107769117
Ireland percent of total:  0.3316196686460686
West Midlands Region percent of total:  0.07667136593504663
South Region percent of total:  0.004010021401389265


From above we can see that the missing values within the IMD Band variable come from the Northern Region and Ireland.

next we need to figure out what to do with the missing values

In [7]:
print(df.imd_band.value_counts(),"\n\n")

print(df[(df.region=='North Region') | (df.region=='Ireland')].imd_band.value_counts())

40-50%     308571
20-30%     293339
50-60%     287071
30-40%     267817
70-80%     263817
90-100%    260616
60-70%     252418
10-20      234069
0-10%      211857
80-90%     195343
Name: imd_band, dtype: int64 


0-10%     28577
10-20     25649
70-80%    24739
60-70%    16246
30-40%    14511
40-50%    11070
20-30%     7659
80-90%     7324
50-60%      370
Name: imd_band, dtype: int64


If we cared enough about filling in these missing values, we could calculate the most likely imd band per region and fill the missing values to match this distribution. However since the missing values only account for less than 8% of the total data, we decided to just drop all the rows containing missing values, as data requirements are of no concern to us

In [14]:
df.dropna(inplace=True)
df.shape

(2524332, 25)

### Outliers

### One Hot Encoding

In [9]:
df.dtypes

code_module                    object
code_presentation              object
id_student                      int64
gender                         object
region                         object
highest_education              object
imd_band                       object
age_band                       object
num_of_prev_attempts            int64
studied_credits                 int64
disability                     object
final_result                   object
id_site                         int64
date_x                          int64
sum_click                       int64
id_assessment                 float64
date_submitted                float64
is_banked                     float64
score                         float64
assessment_type                object
date_y                        float64
weight                        float64
module_presentation_length      int64
date_registration             float64
activity_type                  object
dtype: object

In [10]:
# View unique values in each column
# for col in df:
#   print(df[col].unique())

In [11]:
#Creating variables for indexing continuous and categorical variables
conCol = ['id_student', 'id_site', 'date_x', 'sum_click', 'num_of_prev_attempts', 
          'studied_credits', 'id_assessment ', 'date_submitted', 'is_banked', 'score', 'date_y', 'weight']

catCol = ['code_module', 'code_presentation', 'gender', 'region', 'highest_education', 
          'imd_band', 'age_band', 'disability', 'final_result', 'assessment_type' ]

In [12]:
%%time
#Declaring Scalar object
scl_obj = StandardScaler()

#One hot encoding of ALL categorical variables
OneHotDF = pd.get_dummies(df[catCol],drop_first=False)

#Scaling non-encoded data
conVar = df.select_dtypes(exclude='object')
colnames = pd.Series(conVar.columns)
conVarScaled = scl_obj.fit_transform(conVar)
conVarScaled = pd.DataFrame(data = conVarScaled, columns= colnames)

#Combining with continuous variables from cleaned dataset
OneHotDF = OneHotDF.reset_index()
OneHotDF.pop('index')
OneHotDF = pd.concat([conVarScaled,OneHotDF], axis = 1)
OneHotDF.head()

#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/01.%20Pandas.ipynb


CPU times: user 2.79 s, sys: 964 ms, total: 3.75 s
Wall time: 3.85 s


,id_student,num_of_prev_attempts,studied_credits,id_site,date_x,sum_click,id_assessment,date_submitted,is_banked,score,...,age_band_55<=,disability_N,disability_Y,final_result_Distinction,final_result_Fail,final_result_Pass,final_result_Withdrawn,assessment_type_CMA,assessment_type_Exam,assessment_type_TMA
0,-1.069929,-0.322379,1.148159,1.362644,-1.523433,-0.346119,-0.408612,-1.386095,-0.053341,-0.10248,...,0,1,0,0,0,0,1,1,0,0
1,-1.069929,-0.322379,1.148159,1.364762,-1.523433,-0.346119,-0.408612,-1.386095,-0.053341,-0.10248,...,0,1,0,0,0,0,1,1,0,0
2,-1.069929,-0.322379,1.148159,1.362684,-1.287424,-0.346119,-0.408612,-1.386095,-0.053341,-0.10248,...,0,1,0,0,0,0,1,1,0,0
3,-1.069929,-0.322379,1.148159,1.362684,-1.287424,-0.225184,-0.408612,-1.386095,-0.053341,-0.10248,...,0,1,0,0,0,0,1,1,0,0
4,-1.069929,-0.322379,1.148159,1.362644,-1.287424,0.258553,-0.408612,-1.386095,-0.053341,-0.10248,...,0,1,0,0,0,0,1,1,0,0


# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #